In [1]:
import meep as mp
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
from numpy import cos, sin
import json
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
from scipy.io import FortranFile
import struct

# create geometric and write it to a file

generate random numbers

In [2]:
np.random.seed(15)

num_crystal = 190

a = 5. # shape

size_cell_l = [2, 2, 2]
size_solid_l = [1, 1, 1]
size_crystal_base = [0.1, 0.1, 0.1]
size_crystal_l = np.random.weibull(a, (num_crystal, 3))
for i in range(3):
    size_crystal_l[:, i] *= size_crystal_base[i]

size_cell = mp.Vector3(*size_cell_l)
size_solid = mp.Vector3(*size_solid_l)

size_crystal = []
for i in range(num_crystal):
    size_crystal.append(mp.Vector3(*size_crystal_l[i, :]))

solid_region = mp.Block(size_solid, 
                    center = mp.Vector3(0, 0, 0),
                    material=mp.Medium(epsilon=1))

mean= (0, 0, 0)
cov = [[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.1]]
loc = np.random.multivariate_normal(mean, cov, (num_crystal))
loc = np.random.uniform(-size_solid_l[0]/2, size_solid_l[0]/2, (num_crystal, 3))
theta_x = np.random.uniform(0, 2*np.pi, num_crystal)
theta_y = np.random.uniform(0, 2*np.pi, num_crystal)
theta_z = np.random.uniform(0, 2*np.pi, num_crystal)

Voronoi tesse

In [ ]:
def plot_2d_v(points):
    num_div = 5
    # points = np.zeros((num_div**2, 2)

    index = 0
    for i in range(num_div):
        for j in range(num_div):
            points[index, :] = [i, j]
            index += 1


    from scipy.spatial import Voronoi, voronoi_plot_2d
    vor = Voronoi(points)
    import matplotlib.pyplot as plt
    voronoi_plot_2d(vor)
    plt.show()
    # print(vor.regions)

In [12]:
R = np.empty((num_crystal, 3, 3))

Rx_matrix = np.empty((num_crystal, 3, 3))
Ry_matrix = np.empty((num_crystal, 3, 3))
Rz_matrix = np.empty((num_crystal, 3, 3))

for i in range(num_crystal):
    Rx_matrix[i, :, :] = np.array([[1, 0, 0],
                   [0, cos(theta_x[i]), -sin(theta_x[i])], 
                  [0, sin(theta_x[i]), cos(theta_x[i])]])
    
    Ry_matrix[i, :, :] = np.array([[cos(theta_y[i]), 0, sin(theta_y[i])], 
                  [0, 1, 0],
                  [-sin(theta_y[i]), 0, cos(theta_y[i])]])
    
    Rz_matrix[i, :, :] = np.array([[cos(theta_z[i]), -sin(theta_z[i]), 0],
                 [sin(theta_z[i]), cos(theta_z[i]), 0],
                 [0, 0, 1]])

    R[i, :, :] = np.matmul(np.matmul(Ry_matrix[i, :, :], Rx_matrix[i, :, :]), Rz_matrix[i, :, :])


og_x = np.array([[1, 0, 0] for i in range(num_crystal)])
og_y = np.array([[0, 1, 0] for i in range(num_crystal)])
og_z = np.array([[0, 0, 1] for i in range(num_crystal)])

Rx_vector = np.empty((num_crystal, 3))
Ry_vector = np.empty((num_crystal, 3))
Rz_vector = np.empty((num_crystal, 3))

for i in range(num_crystal):
    Rx_vector[i, :] = np.matmul(R[i, :, :], og_x[i, :])
    Ry_vector[i, :] = np.matmul(R[i, :, :], og_y[i, :])
    Rz_vector[i, :] = np.matmul(R[i, :, :], og_z[i, :])

geometry = [solid_region,]

for i in range(num_crystal):
    if (np.abs(loc[i, 0]) < size_solid[0] - size_crystal_base[0]/2 and 
    np.abs(loc[i, 1]) < size_solid[1] - size_crystal_base[1]/2 and 
    np.abs(loc[i, 2]) < size_solid[2] - size_crystal_base[2]/2):
        geometry.append(mp.Block(
            size_crystal[i],
            center = mp.Vector3(loc[i, 0], loc[i, 1], loc[i, 2]),
            e1 = Rx_vector[i, :],
            e2 = Ry_vector[i, :],
            e3 = Rz_vector[i, :],
            material=mp.Medium(epsilon=10.5)))



In [13]:
export_geo = True

if export_geo:
    file_name = 'geometry.peter'
    to_write = [num_crystal, size_solid_l, size_crystal_l, loc, theta_x, theta_y, theta_z]
    for i in range(len(to_write)):
        if type(to_write[i]) is not int and type(to_write[i]) is not list:
            to_write[i] = to_write[i].tolist()
    with open(file_name, 'w') as f:
        json.dump(to_write, f)

#\\ad.monash.edu\home\User045\dche145\Documents\Abaqus\geometry_shapes
#\\Client\D$\source\working_with_meep

helper functions

In [14]:
def closest_node(node, nodes):
    # nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)

    
def my_mat(coord):
    if coord[0] <  0.25 and coord[0] > -0.25 :
        return mp.Medium(epsilon=10.5)
    else:
        return mp.Medium(epsilon=1)

def my_eps(coord):
    acoord = np.abs(coord)
    if (acoord[0] < 0.5 and acoord[1] < 0.5 and acoord[2] < 0.5 ):
        return parts_eps[closest_node([coord[0],coord[1],coord[2]], points)]
    else:
        return 1.0

def my_part_ass(coord):
    if (acoord[0] < 0.5 and acoord[1] < 0.5 and acoord[2] < 0.5 ):
        return parts_ass[closest_node([coord[0],coord[1],coord[2]], points)]
    else:
        return 1.0

def index2coord(index, size_arr, size_geo):
    index = (index/size_arr - 0.5)*size_geo
    return index

In [15]:
num_div = 20
num_seed = num_div**3
points = np.zeros((num_div**3, 3))

p_range = np.array([1.0, 1.0, 1.0])
p_div = p_range/num_div
num_parts = 2
eps_val = [1, 10]

# vor = Voronoi(points)

# assigning points and seeds
random_ass = True
if random_ass:
    np.random.seed(15)
    points = np.random.rand(num_seed, 3) - 0.5
    parts_ass = np.random.randint(0, num_parts, (num_seed))
else:
    index = 0
    for i in range(num_div):
        for j in range(num_div):
            for k in range(num_div):
                points[index, :] = [i*p_div[0] - p_range[0]/2, j*p_div[1] - p_range[1]/2, k*p_div[2] - p_range[2]/2]
                index += 1
    # this will produce checker pattern
    parts_ass = np.array([1 if i%2 else 0 for i in range(num_seed)])

parts_eps = [eps_val[parts_ass[i]] for i in range(num_seed)]



In [20]:
out_test = np.array((
    (1,2,3),
    (4,5,6),
    (7,8,9)
)
)
out_test = out_test.transpose()
out_test = out_test.astype('<f8')
with open('../abaqus_working_space/abaqus_out/test.geo', 'wb') as f:
        out_test.tofile(f)

In [11]:
#output geometric files
update_geofile = True
if update_geofile:
    size_out_geo = [100, 100, 100]
    out_geo = np.zeros((size_out_geo))
    for i in range(size_out_geo[0]):
        for j in range(size_out_geo[1]):
            for k in range(size_out_geo[2]):
                coord = index2coord(np.array((i,j,k),dtype=float), size_out_geo, size_cell_l)
                out_geo[i,j,k] = parts_ass[closest_node(coord, points)]
    out_geo = out_geo.astype('<f4')
    out_geo = out_geo.transpose()
    with open('../abaqus_working_space/abaqus_out/voronoi_geo.peter', 'wb') as f:
        out_geo.tofile(f)

In [ ]:
#construct the geometry
pml_layers = [mp.PML(0.3)]

source_pad = 0.25
source = [mp.Source(mp.ContinuousSource(wavelength=2*(11**0.5), width=20),
                   component=mp.Ez,
                   center=mp.Vector3(0.75, 0, 0),
                   size=mp.Vector3(0, 0.1, 0.1))]


sim = mp.Simulation(resolution=100,
                    cell_size=size_cell,
                    boundary_layers=pml_layers,
                    sources = source,
                    epsilon_func = my_eps)

ez_straight_waveg = []
def f(sim):
    ez_straight_waveg.append(sim.get_efield_z())    

sim.run(mp.at_every(1, f), until=200)
ez_straight_waveg = np.array(ez_straight_waveg)
ez_straight_waveg = ez_straight_waveg.transpose()
# ez_straight_waveg = ez_straight_waveg.astype('<f4')

with open('one_cube_3d.bin', 'wb') as f:
    ez_straight_waveg.tofile(f)
# plt.imshow(ez_straight_waveg[1])

Create the visualization for visual inspection

In [82]:
sim.init_sim()
eps_data = sim.get_epsilon()

show_3d_geo = True
if show_3d_geo:
    from mayavi import mlab
    s = mlab.contour3d(eps_data, colormap="YlGnBu")
    mlab.show()

-----------
Initializing structure...
time for choose_chunkdivision = 0.000535965 s
Working in 3D dimensions.
Computational cell is 2 x 2 x 2 with resolution 100
subpixel-averaging is 67.7694% done, 1.90628 s remaining
subpixel-averaging is 67.7694% done, 1.90443 s remaining
subpixel-averaging is 66.4754% done, 2.01866 s remaining
subpixel-averaging is 65.8285% done, 2.07667 s remaining
subpixel-averaging is 64.858% done, 2.16802 s remaining
subpixel-averaging is 64.6963% done, 2.18635 s remaining
subpixel-averaging is 13.926% done, 25.2267 s remaining
subpixel-averaging is 14.8941% done, 23.668 s remaining
subpixel-averaging is 15.6761% done, 21.7632 s remaining
subpixel-averaging is 16.6814% done, 20.904 s remaining
subpixel-averaging is 17.4633% done, 18.9879 s remaining
subpixel-averaging is 18.2453% done, 18.0002 s remaining
subpixel-averaging is 19.0272% done, 17.1777 s remaining
subpixel-averaging is 19.8092% done, 16.3566 s remaining
subpixel-averaging is 20.5911% done, 15.5932

cheeky run of the geometry

In [83]:
sim.run(mp.at_beginning(mp.output_epsilon), 
       mp.to_appended("tesselation_out", mp.at_every(0.6, mp.output_efield_z)),
       until=20)

s/step
Meep progress: 6.86/20.0 = 34.3% done in 388.4s, 744.0s to go
on time step 1372 (time=6.86), 0.267025 s/step
Meep progress: 6.94/20.0 = 34.7% done in 392.7s, 738.9s to go
on time step 1388 (time=6.94), 0.266034 s/step
Meep progress: 7.0200000000000005/20.0 = 35.1% done in 396.9s, 733.9s to go
on time step 1404 (time=7.02), 0.265915 s/step
Meep progress: 7.1000000000000005/20.0 = 35.5% done in 401.2s, 728.9s to go
on time step 1420 (time=7.1), 0.265758 s/step
Meep progress: 7.18/20.0 = 35.9% done in 405.4s, 723.9s to go
on time step 1436 (time=7.18), 0.266002 s/step
Meep progress: 7.23/20.0 = 36.1% done in 409.6s, 723.4s to go
on time step 1446 (time=7.23), 0.412781 s/step
Meep progress: 7.3100000000000005/20.0 = 36.5% done in 413.8s, 718.4s to go
on time step 1462 (time=7.31), 0.266474 s/step
Meep progress: 7.385/20.0 = 36.9% done in 417.8s, 713.7s to go
on time step 1477 (time=7.385), 0.267792 s/step
Meep progress: 7.465/20.0 = 37.3% done in 422.1s, 708.8s to go
on time step 14

visulization of the output field

In [38]:
# ez_data = sim.get_array(center=mp.Vector3(), size=size_cell, component=mp.Ez)
# print(ez_data.shape)
get_gif_out = False

if get_gif_out:
    !h5ls ez.h5
    !mkdir temp
    !mv eps-000000.00.h5 temp/eps-000000.00.h5
    !mv ez.h5 temp/ez.h5
    import os
    os.chdir('temp')
    !h5topng -z 20 -t 0:332 -R -Zc dkbluered -a yarg -A eps-000000.00.h5 ez.h5
    !convert ez.t*.png ez.gif
    os.chdir('..')

(100, 100, 100)
ez                       Dataset {100, 100, 100, 333/Inf}
h5topng error: invalid colormap file
